In [148]:
import geopandas as gpd
import pandas as pd
import os
import parcelfunks
import numpy as np
from scipy import stats
import shutil
from glob import glob
import importlib
import numpy as np
import importlib

In [2]:
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
#CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\ColoradoInventory.csv', dtype={'STATE':str,'COUNTY':str})
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv')) # need to get this to work
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [147]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
unionDF = pd.DataFrame()
joinDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,fips+'double.gpkg')):
        union = gpd.read_file(os.path.join(path, fips+'double.gpkg'),layer='MH_parc_blk_union')
        unionDF = pd.concat([unionDF,union])
        join = gpd.read_file(os.path.join(path, fips+'double.gpkg'),layer='MH_parcels')
        joinDF = pd.concat([joinDF,join])        
unionDF.to_file(os.path.join(co_22_dir,'Colorado_Final_double.gpkg'),layer='Colorado_Final_union')
#joinDF['polyLen_3'] = joinDF.apply(lambda row: geomLen(row.geometry), axis=1)
#joinDF['geomZscore_3'] = np.abs(stats.zscore(joinDF['polyLen2']))
joinDF.to_file(os.path.join(co_22_dir, 'Colorado_Final_double.gpkg'),layer='Colorado_Final_MH_parcels')

In [ ]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
final_near_join = gpd.read_file(os.path.join(co_22_dir, 'Colorado_Final_multi100_less5.gpkg'), layer='Colorado_Final_MH_parcels')
final_near_join.to_csv(os.path.join(co_22_dir, 'Colorado_Final_multi100_less5_parcels.csv'))

In [ ]:
for fips, path in zip(fipsList,parcelsPaths):
    if os.path.exists(os.path.join(path,fips+'multi100_less5.gpkg')):
        os.remove((os.path.join(path,fips+'multi100_less5.gpkg')))

In [64]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [17]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
counties_dir = os.path.join(co_22_dir,'counties')
mesa_dir = os.path.join(counties_dir,'08041')

In [29]:
mesaBuildings = gpd.read_file(os.path.join(mesa_dir,'08041_buildings.shp')).to_crs(crs='ESRI:102003')
mesaMHPs = gpd.read_file(os.path.join(mesa_dir,'08041_mhps_OG.gpkg'),layer='08041_mhps_OG_prepped').to_crs(crs='ESRI:102003')

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08041\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [41]:
mesaParcels = gpd.read_file(os.path.join(mesa_dir,'parcels.shp')).to_crs(crs='ESRI:102003')


c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08041\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [65]:
mesaParcelSelect= parcelfunks.parcelPreSelect(mesaParcels,mesaBuildings)

In [66]:
cols = mesaParcelSelect.columns

In [63]:
mesaParcelFiltered.to_file(os.path.join(mesa_dir,'08041multi100_less5.gpkg'),layer='preSelect')

In [67]:
mesaParcelSelect['ID'] = mesaParcelSelect.index

In [73]:
mhps_parcels_near = gpd.sjoin_nearest(mesaMHPs,mesaParcelSelect,max_distance=100, distance_col='distances')

In [74]:
mhps_parcels_near.drop(cols,axis=1,inplace=True)

In [76]:
mhps_parcels_near.columns

Index(['MHPID', 'MH_NAME', 'MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP',
       'MH_STATUS', 'MH_COUNTY_FIPS', 'MH_LATITUDE', 'MH_LONGITUDE',
       'index_right', 'ID', 'distances'],
      dtype='object')

In [78]:
print(len(mesaMHPs))
print(len(mhps_parcels_near))

53
48


In [117]:
merged = mesaParcelSelect.merge(mhps_parcels_near, on='ID')

In [118]:
merged.drop(['index_right'],axis=1,inplace=True)

In [82]:
merged.to_file(os.path.join(mesa_dir,'08041multi100_less5.gpkg'),layer='reverseJoin')

In [119]:
merged = merged.sort_values(['ID','distances']).drop_duplicates(subset=['ID'], keep='first')

In [121]:
len(merged)

46

In [122]:
len(mesaParcelSelect)

462

In [123]:
IDlist = merged['ID'].to_list()

In [124]:
IDlist[0]

159

In [125]:
mesaParcelSelect['IDcheck'] = mesaParcelSelect['ID'].isin(IDlist)

In [126]:
unmatched = mesaParcelSelect.loc[mesaParcelSelect['IDcheck']==False]

In [128]:
len(unmatched) + len(merged) == len(mesaParcelSelect)

True

In [130]:
secondJoin = gpd.sjoin_nearest(unmatched, mesaMHPs, max_distance=150.0, distance_col='distances')

In [133]:
secondJoin.drop('index_right',axis=1,inplace=True)

In [134]:
secondJoin.to_file(os.path.join(mesa_dir,'08041multi100_less5.gpkg'),layer='secondJoin')

In [138]:
concatted = pd.concat([merged,secondJoin])

In [139]:
concatted.to_file(os.path.join(mesa_dir,'08041multi100_less5.gpkg'),layer='doubleJoin')

In [137]:
secondJoin.columns

Index(['APN', 'APN2', 'geometry', 'Sum_Within', 'mnBlgArea', 'intLen',
       'intZscore', 'extLen1', 'extZscore1', 'ID', 'IDcheck', 'MHPID',
       'MH_NAME', 'MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP', 'MH_STATUS',
       'MH_COUNTY_FIPS', 'MH_LATITUDE', 'MH_LONGITUDE', 'distances'],
      dtype='object')

In [145]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [141]:
mesa_dir

'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\counties\\08041'

In [146]:
parcelfunks.parcelMHPJoin(mesa_dir)

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08041\parcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


figuring out warning stuff

In [153]:
mesaParcels = gpd.read_file(os.path.join(mesa_dir,'newparcels.shp')).to_crs(crs='ESRI:102003')

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08041\newparcels.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [150]:
mesa_dir

'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\counties\\08041'